# **Hands-on Lab: Interactive Visual Analytics with Folium**

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.
    In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.

In [9]:
!pip install folium
!pip install pandas

In [14]:
import folium
import pandas as pd

In [16]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Task 1: Mark all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates.
The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.

In [21]:
import requests
from io import StringIO

url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = requests.get(url)
csv_data = response.text
spacex_df = pd.read_csv(StringIO(csv_data))

Now, you can take a look at what are the coordinates for each site.

In [24]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [28]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,

In [31]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.
Now, let's add a circle for each launch site in data frame `launch_sites`.
*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map.
An example of folium.Circle:
`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`
An example of folium.Marker:
`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`

In [34]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
import folium
from folium import DivIcon

# Sample data for launch sites; replace this with your actual data.
# Ensure this contains columns: ['LaunchSite', 'Latitude', 'Longitude']
launch_sites = [
    {'LaunchSite': 'Cape Canaveral', 'Latitude': 28.3922, 'Longitude': -80.6077},
    {'LaunchSite': 'Vandenberg Space Force Base', 'Latitude': 34.7336, 'Longitude': -120.5734},
    {'LaunchSite': 'Kennedy Space Center', 'Latitude': 28.5721, 'Longitude': -80.6480}
]

# Create a Folium map centered around the average coordinates
map_center = [28.3922, -80.6077]  # Example center; adjust as necessary
site_map = folium.Map(location=map_center, zoom_start=6)

# Loop through the launch sites and add circles and markers
for site in launch_sites:
    coordinate = (site['Latitude'], site['Longitude'])
    
    # Add a Circle
    folium.Circle(
        location=coordinate,
        radius=1000,  # Radius in meters
        color='#000000',
        fill=True,
        fill_color='#000000',
        fill_opacity=0.3  # Change opacity for visibility
    ).add_to(site_map)
    
    # Add a Marker
    folium.map.Marker(
        location=coordinate,
        icon=DivIcon(icon_size=(20, 20), icon_anchor=(0, 0),
                     html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site['LaunchSite'])
    ).add_to(site_map)

# Display the map
site_map

Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.

# Task 2: Mark the success/failed launches for each site on the map

In [52]:
print("Column names in spacex_df:", spacex_df.columns)

# Initialize a MarkerCluster for better visualization when markers are close
marker_cluster = MarkerCluster().add_to(site_map)

# Assign marker colors based on the class (1 for success, 0 for failure)
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Add each launch site with a color-coded marker for success/failure
for _, record in spacex_df.iterrows():
    folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color=record['marker_color']),
        popup=f"Launch Site: {record['Launch Site']}<br>Flight Number: {record['Flight Number']}<br>Outcome: {'Success' if record['class'] == 1 else 'Failed'}"
    ).add_to(marker_cluster)

# Display the map
site_map

Column names in spacex_df: Index(['Flight Number', 'Date', 'Time (UTC)', 'Booster Version', 'Launch Site',
       'Payload', 'Payload Mass (kg)', 'Orbit', 'Customer', 'Landing Outcome',
       'class', 'Lat', 'Long', 'marker_color'],
      dtype='object')


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not

In [55]:
spacex_df.tail(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long,marker_color
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895,green
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895,green
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895,green
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820,green
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820,green
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820,red
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820,red
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820,red
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820,green
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820,red


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`
Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.
Let's first create a `MarkerCluster` object.

In [58]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value

In [61]:
# Step 1: Check the value of `Class` column and create `marker_color` column
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`

In [64]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.

Next, we need to explore and analyze the proximities of launch sites.
Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)

In [68]:
from folium.plugins import MarkerCluster, MousePosition

# Add MousePosition plugin to display coordinates
mouse_position = MousePosition(
    position='topright',
    separator=' | ',
    empty_string='No coordinates',
    num_digits=5,
    prefix='Coordinates:',
    lat_formatter="function(num) {return L.Util.formatNum(num, 5);}",
    lng_formatter="function(num) {return L.Util.formatNum(num, 5);}",
)
mouse_position.add_to(site_map)

# Initialize a MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add markers for each launch site with color-coded icons based on success/failure
for _, record in spacex_df.iterrows():
    folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color=record['marker_color']),
        popup=f"Launch Site: {record['Launch Site']}<br>Flight Number: {record['Flight Number']}<br>Outcome: {'Success' if record['class'] == 1 else 'Failed'}"
    ).add_to(marker_cluster)

# Display the map
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [78]:
import folium
from folium.plugins import MarkerCluster, MousePosition
import pandas as pd
import requests
from io import StringIO
from math import radians, cos, sin, sqrt, atan2

# Function to calculate the distance using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    # Radius of Earth in kilometers (mean radius)
    R = 6371.0
    return R * c

# Load SpaceX data
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"
response = requests.get(url)
spacex_df = pd.read_csv(StringIO(response.text))

# Define the base map centered around the average coordinates in the data
site_map = folium.Map(location=[spacex_df['Lat'].mean(), spacex_df['Long'].mean()], zoom_start=5)

# Add MousePosition to get coordinates for any point on the map
mouse_position = MousePosition(
    position='topright',
    separator=' | ',
    empty_string='No coordinates',
    num_digits=5,
    prefix='Coordinates:',
    lat_formatter="function(num) {return L.Util.formatNum(num, 5);}",
    lng_formatter="function(num) {return L.Util.formatNum(num, 5);}",
)
mouse_position.add_to(site_map)

# Select a launch site to calculate the distance to the coastline
launch_site = spacex_df.iloc[0]  # Selecting the first launch site as an example
launch_site_coords = (launch_site['Lat'], launch_site['Long'])

# Manually set the closest coastline coordinates based on MousePosition output (Example values here)
coastline_coords = (28.56341, -80.56734)  # Replace these with actual coordinates from MousePosition

# Calculate the distance
distance_to_coastline = calculate_distance(launch_site_coords[0], launch_site_coords[1], coastline_coords[0], coastline_coords[1])

# Mark the coastline point on the map
folium.Marker(
    location=coastline_coords,
    icon=folium.DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12px; color:#d35400;"><b>{distance_to_coastline:.2f} km</b></div>',
    ),
    tooltip="Closest Coastline"
).add_to(site_map)

# Display the map
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point

In [80]:
# Draw a PolyLine between the launch site and coastline point
folium.PolyLine(
    locations=[launch_site_coords, coastline_coords],
    color='blue',
    weight=2,
    tooltip="Distance to Coastline"
).add_to(site_map)

# Display the map
site_map